<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/knn-model/machine-learning/knn-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dagshub --quiet
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/

In [1]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA

In [37]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\facial-emotion-recognition-augmented')
LABELS = os.listdir(PATH)

In [6]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=927c6126-b759-4dfc-9f88-f67783eba6ab&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=7ba60dfda919f06be1a830a0a2c66183522ce73025979ffa7130996941a18df0




Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/face-emotion-recognition"

Repository IdjiotSandwiches/face-emotion-recognition initialized!

In [38]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/128)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0
FLOATING_POINT = cv.CV_64F
IMAGE_SIZE = (128,128)
N_COMPONENTS = 10
BLUR = (5,5)

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

KERNELS = [cv.getGaborKernel(**gabor_params, theta=theta) for theta in THETA_RANGE]

In [30]:
def open_dataset(dir):
  images, labels = [], []

  for i, label in enumerate(LABELS):
    path = f'{dir}/{label}'
    for j, img_path in enumerate(os.listdir(path)):
      img = cv.imread(f'{path}/{img_path}', cv.IMREAD_GRAYSCALE)
      img = cv.resize(img, IMAGE_SIZE)
      img = cv.GaussianBlur(img,BLUR,0)
      img = cv.equalizeHist(img)
      img = img / 255.0
      img = gabor_filter(img)

      img = img.reshape(img.shape[0],-1)
      pca = PCA(n_components=N_COMPONENTS)
      img = pca.fit_transform(img)

      images.append(img.reshape(-1))
      labels.append(i)

  return np.array(images), np.array(labels)

In [9]:
def gabor_filter(img):
  return np.array([cv.filter2D(img, FLOATING_POINT, kernel) for kernel in KERNELS])

In [34]:
def save_filtered_img(images, labels):
  folder_path = f'C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs'
  os.makedirs(folder_path, exist_ok=True)

  np.save(f'{folder_path}/images.npy', images)
  np.save(f'{folder_path}/labels.npy', labels)

  print('Ok!')

In [35]:
from joblib import Parallel, delayed

def process_image(img_path, label, dir):
    """
    Process a single image: read, preprocess, and apply PCA.
    """
    path = f'{dir}/{label}'
    img = cv.imread(f'{path}/{img_path}', cv.IMREAD_GRAYSCALE)
    img = cv.resize(img, IMAGE_SIZE)
    img = cv.GaussianBlur(img, (5, 5), 0)
    img = cv.equalizeHist(img)
    img = img / 255.0
    img = gabor_filter(img)

    img = img.reshape(img.shape[0], -1)
    pca = PCA(n_components=N_COMPONENTS)
    img = pca.fit_transform(img)

    return img.reshape(-1), label


def open_dataset(dir):
    """
    Open dataset, load images, preprocess, and return images and labels using parallel processing.
    """
    # Use Parallel from joblib to process images in parallel
    results = Parallel(n_jobs=4)(delayed(process_image)(img_path, label, dir)
                                   for label in LABELS
                                   for img_path in os.listdir(f'{dir}/{label}'))

    # Unzip the results into images and labels
    images, labels = zip(*results)

    return np.array(images), np.array(labels)

In [39]:
images, labels = open_dataset(PATH)
save_filtered_img(images, labels)

Ok!


## **Testing**

In [40]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\labels.npy')

N_NEIGHBORS = 3
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'
TEST_SIZE = 0.5
RANDOM_STATE = 42
N_SPLITS = 20
SHUFFLE = True

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

calib_params = {
  'test_size': TEST_SIZE,
  'random_state': RANDOM_STATE
}

kfold_params = {
    'n_splits': N_SPLITS,
    'shuffle': SHUFFLE
}

skf = StratifiedKFold(**kfold_params)
i = 0

for train_idx, test_idx in skf.split(images, labels):
  X_train, X_test = images[train_idx], images[test_idx]
  y_train, y_test = labels[train_idx], labels[test_idx]

  X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params)

  model = KNeighborsClassifier(**model_params)
  model.fit(X_train, y_train)

  calib_model = CalibratedClassifierCV(model, cv="prefit")
  calib_model.fit(X_calib, y_calib)

  predict = calib_model.predict(X_test)

  metrics = {
    'accuracy': accuracy_score(y_test, predict),
    'precision': precision_score(y_test, predict, average='macro'),
    'recall': recall_score(y_test, predict, average='macro'),
    'f1': f1_score(y_test, predict, average='macro'),
  }

  print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
  print(classification_report(y_test, predict))

  mlflow.set_experiment("KNN_Model using PCA with more data")
  mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow")

  with mlflow.start_run(run_name=f'KNN StratifiedKFold={i}'):
    mlflow.log_params(gabor_params)
    mlflow.log_params(calib_params)
    mlflow.log_params(model_params)
    mlflow.log_params(kfold_params)
    mlflow.log_param('floating_point', FLOATING_POINT)
    mlflow.log_param('image_size', IMAGE_SIZE)
    mlflow.log_param('PCA_n_components', N_COMPONENTS)
    mlflow.log_param('gaussian_blur', BLUR)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model=calib_model,
        artifact_path='KNN Model',
        input_example=X_train[:1]
    )
  i = i + 1

Accuracy: 0.5771812080536913
Precision: 0.5863184743105132
Recall: 0.5792257481050368
F1 Score: 0.578743758117012
              precision    recall  f1-score   support

       angry       0.66      0.56      0.60        52
     disgust       0.65      0.59      0.62        37
        fear       0.57      0.65      0.61        43
       happy       0.58      0.48      0.52        46
     neutral       0.57      0.62      0.60        40
         sad       0.40      0.55      0.47        38
    surprise       0.68      0.60      0.63        42

    accuracy                           0.58       298
   macro avg       0.59      0.58      0.58       298
weighted avg       0.59      0.58      0.58       298



2024/12/13 00:34:40 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model using PCA with more data' does not exist. Creating a new experiment.


🏃 View run KNN StratifiedKFold=0 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/318dc15941e04e1f825d2008bb9fcc1b
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.6040268456375839
Precision: 0.6137680295731889
Recall: 0.6027455622409894
F1 Score: 0.6044179019667666
              precision    recall  f1-score   support

       angry       0.73      0.63      0.68        52
     disgust       0.62      0.65      0.63        37
        fear       0.70      0.70      0.70        43
       happy       0.46      0.52      0.49        46
     neutral       0.47      0.60      0.53        40
         sad       0.64      0.47      0.55        38
    surprise       0.68      0.64      0.66        42

    accuracy                           0.60       298
   macro avg       0.61      0.60      0.60       298
weighted avg       0.62      0.60      0.61       298



🏃 View run KNN StratifiedKFold=1 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/fc2eba2ca93346b9a0a3bafa19ec4a0f
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.5906040268456376
Precision: 0.5953138654986062
Recall: 0.5979342658474766
F1 Score: 0.5917723117509611
              precision    recall  f1-score   support

       angry       0.63      0.46      0.53        52
     disgust       0.63      0.70      0.67        37
        fear       0.65      0.70      0.67        43
       happy       0.50      0.48      0.49        46
     neutral       0.51      0.70      0.59        40
         sad       0.61      0.53      0.56        38
    surprise       0.63      0.62      0.63        42

    accuracy                           0.59       298
   macro avg       0.60      0.60      0.59       298
weighted avg       0.60      0.59      0.59       298



🏃 View run KNN StratifiedKFold=2 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/a367501025534cadb599ef6d59e8bb28
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.6006711409395973
Precision: 0.6077811730752908
Recall: 0.6115213830862279
F1 Score: 0.603909074858592
              precision    recall  f1-score   support

       angry       0.64      0.40      0.49        52
     disgust       0.71      0.65      0.68        37
        fear       0.61      0.63      0.62        43
       happy       0.50      0.52      0.51        46
     neutral       0.54      0.68      0.60        40
         sad       0.62      0.74      0.67        38
    surprise       0.64      0.67      0.65        42

    accuracy                           0.60       298
   macro avg       0.61      0.61      0.60       298
weighted avg       0.61      0.60      0.60       298



...

🏃 View run KNN StratifiedKFold=15 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/6626d61174c94a599cc8cba7c2dfc22a
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.5771812080536913
Precision: 0.5871150992384851
Recall: 0.5850606333025727
F1 Score: 0.578731834367904
              precision    recall  f1-score   support

       angry       0.61      0.48      0.54        52
     disgust       0.50      0.68      0.57        37
        fear       0.64      0.58      0.61        43
       happy       0.59      0.48      0.53        46
     neutral       0.58      0.65      0.61        40
         sad       0.48      0.66      0.56        38
    surprise       0.71      0.57      0.63        42

    accuracy                           0.58       298
   macro avg       0.59      0.59      0.58       298
weighted avg       0.59      0.58      0.58       298



🏃 View run KNN StratifiedKFold=16 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/1d097603262347ccb33d78b2495c71bd
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.5570469798657718
Precision: 0.5627396863111149
Recall: 0.5647481334166449
F1 Score: 0.5576287327229267
              precision    recall  f1-score   support

       angry       0.70      0.50      0.58        52
     disgust       0.62      0.62      0.62        37
        fear       0.54      0.60      0.57        43
       happy       0.38      0.33      0.35        46
     neutral       0.63      0.55      0.59        40
         sad       0.50      0.68      0.58        38
    surprise       0.56      0.67      0.61        42

    accuracy                           0.56       298
   macro avg       0.56      0.56      0.56       298
weighted avg       0.56      0.56      0.55       298



🏃 View run KNN StratifiedKFold=17 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/b4a661803d104e7ba9ac17d5e0cffb70
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.610738255033557
Precision: 0.6198721743550152
Recall: 0.6194920851954614
F1 Score: 0.6146582813217503
              precision    recall  f1-score   support

       angry       0.69      0.52      0.59        52
     disgust       0.60      0.76      0.67        37
        fear       0.69      0.67      0.68        43
       happy       0.46      0.41      0.44        46
     neutral       0.82      0.78      0.79        40
         sad       0.43      0.58      0.49        38
    surprise       0.65      0.62      0.63        42

    accuracy                           0.61       298
   macro avg       0.62      0.62      0.61       298
weighted avg       0.62      0.61      0.61       298



🏃 View run KNN StratifiedKFold=18 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/dca51635301c405883a0ce742d145761
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1
Accuracy: 0.5436241610738255
Precision: 0.5526464971953694
Recall: 0.5548842022138508
F1 Score: 0.5422368165131649
              precision    recall  f1-score   support

       angry       0.67      0.35      0.46        52
     disgust       0.56      0.68      0.61        37
        fear       0.74      0.72      0.73        43
       happy       0.47      0.39      0.43        46
     neutral       0.50      0.70      0.58        40
         sad       0.48      0.53      0.50        38
    surprise       0.46      0.52      0.49        42

    accuracy                           0.54       298
   macro avg       0.55      0.55      0.54       298
weighted avg       0.56      0.54      0.54       298



🏃 View run KNN StratifiedKFold=19 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1/runs/da1d7bc74f9e4181986fa85c08d7f086
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/1


## **Testing images shape**

In [ ]:
images = []
# img = cv.imread(f'facial-emotion-recognition-augmented/disgust/disgust_1022.png', cv.IMREAD_GRAYSCALE)
img = cv.imread(f'eupho_test.png', cv.IMREAD_GRAYSCALE)
img = cv.resize(img, (128,128))
img = cv.GaussianBlur(img,(5,5),0)
img = cv.equalizeHist(img)
img = img / 255.0
img = gabor_filter(img)
img = np.array(img)
img = img.reshape(img.shape[0],-1)
pca = PCA(n_components=10)
img = pca.fit_transform(img)
img = img.reshape(-1)
images.append(img)
images = np.array(images)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Cumulative explained variance:", np.cumsum(pca.explained_variance_ratio_))

print(images.shape)

Explained variance ratio: [0.42724823 0.25657417 0.13746045 0.08621814 0.03908073 0.02821402
 0.0108493  0.00836474 0.00245178 0.00201102]
Cumulative explained variance: [0.42724823 0.6838224  0.82128286 0.907501   0.94658172 0.97479575
 0.98564505 0.99400978 0.99646157 0.99847258]
(1, 1280)


In [ ]:
predict = model.predict(images)

In [ ]:
for p in predict:
  print(LABELS[p])

neutral


## **Tuning**

### **Without PCA**

In [ ]:
images = np.load('/content/drive/MyDrive/gabor-filtered-imgs/images.npy')
labels = np.load('/content/drive/MyDrive/gabor-filtered-imgs/labels.npy')

N_NEIGHBORS = [3,5,7,9,11]
WEIGHTS = ['uniform', 'distance']
METRIC = ['minkowski', 'euclidean', 'manhattan']
ALGORITHM = ['auto', 'ball_tree', 'kd_tree', 'brute']
TEST_SIZE = 0.6
RANDOM_STATE = 42
N_SPLITS = 15
SHUFFLE = True

calib_params = {
  'test_size': TEST_SIZE,
  'random_state': RANDOM_STATE
}

for n in N_NEIGHBORS:
  for w in WEIGHTS:
    for m in METRIC:
      for a in ALGORITHM:
        model_params = {
          'n_neighbors': n,
          'weights': w,
          'metric': m,
          'algorithm': a
        }

        X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
        X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params)

        model = KNeighborsClassifier(**model_params)
        model.fit(X_train, y_train)

        calib_model = CalibratedClassifierCV(model, cv="prefit")
        calib_model.fit(X_calib, y_calib)

        predict = calib_model.predict(X_test)

        metrics = {
          'accuracy': accuracy_score(y_test, predict),
          'precision': precision_score(y_test, predict, average='macro'),
          'recall': recall_score(y_test, predict, average='macro'),
          'f1': f1_score(y_test, predict, average='macro'),
        }

        print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
        print(classification_report(y_test, predict))

        mlflow.set_experiment("KNN_Model Tuning Model without using PCA")
        mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

        with mlflow.start_run():
          mlflow.log_params(gabor_params)
          mlflow.log_params(calib_params)
          mlflow.log_params(model_params)
          # mlflow.log_params(kfold_params)
          mlflow.log_param('floating_point', FLOATING_POINT)
          mlflow.log_param('image_size', IMAGE_SIZE)
          mlflow.log_param('PCA_n_components', N_COMPONENTS)
          mlflow.log_metrics(metrics)
          mlflow.sklearn.log_model(
              sk_model=calib_model,
              artifact_path='KNN Model',
                input_example=X_train[:1]
          )

Accuracy: 0.2673434856175973
Precision: 0.2725667470879056
Recall: 0.274931070605638
F1 Score: 0.2566980461274672
              precision    recall  f1-score   support

           0       0.33      0.19      0.24       177
           1       0.28      0.44      0.34       153
           2       0.25      0.48      0.33       153
           3       0.25      0.25      0.25       189
           4       0.28      0.13      0.18       182
           5       0.26      0.29      0.27       161
           6       0.26      0.14      0.19       167

    accuracy                           0.27      1182
   macro avg       0.27      0.27      0.26      1182
weighted avg       0.27      0.27      0.25      1182

🏃 View run placid-bat-562 at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/14/runs/6d5a1239cd3d4c0da9fa00e94241104f
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/14
Accuracy: 0.2673434856175973
Precision: 0.2725667470879056
Re

### **Using calibration**

In [ ]:
N_NEIGHBORS = 9
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'

TEST_SIZE = 0.3
CALIB_SIZE = 0.5
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

calib_params = {
    'test_size': CALIB_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)
X_test, X_calib, y_test, y_calib = train_test_split(X_test, y_test, **calib_params)

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

model = KNeighborsClassifier(**model_params)
model.fit(X_train, y_train)

calib_model = CalibratedClassifierCV(model, cv="prefit")
calib_model.fit(X_calib, y_calib)

predict = calib_model.predict(X_test)

metrics = {
  'accuracy': accuracy_score(y_test, predict),
  'precision': precision_score(y_test, predict, average='macro'),
  'recall': recall_score(y_test, predict, average='macro'),
  'f1': f1_score(y_test, predict, average='macro'),
}

print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
print(classification_report(y_test, predict))

mlflow.set_experiment("KNN_Model using calibration")
mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

with mlflow.start_run(run_name=f'KNN sigma=5'):
  mlflow.log_params(gabor_params)
  mlflow.log_params(split_params)
  mlflow.log_param('calib_test_size', calib_params['test_size'])
  mlflow.log_param('calib_random_state', calib_params['random_state'])
  mlflow.log_params(model_params)
  mlflow.log_metrics(metrics)
  mlflow.sklearn.log_model(
      sk_model=calib_model,
      artifact_path='KNN Model',
      input_example=X_train[:1]
  )

Accuracy: 0.41271820448877805
Precision: 0.35505141285443115
Recall: 0.33373441164794554
F1 Score: 0.3324115318106068


2024/12/04 03:16:05 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model using calibration' does not exist. Creating a new experiment.


🏃 View run KNN sigma=5 at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/8/runs/c1b2dcc5917e455fa84e0b57000e5062
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/8


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

## **Prediction result**

In [ ]:
for p, y in zip(predict[:20], y_test[:20]):
  print(f'Predict: {LABELS[p]} | Real: {LABELS[y]}')

Predict: happy | Real: surprise
Predict: surprise | Real: surprise
Predict: surprise | Real: fear
Predict: happy | Real: happy
Predict: happy | Real: sad
Predict: surprise | Real: neutral
Predict: neutral | Real: sad
Predict: happy | Real: happy
Predict: angry | Real: neutral
Predict: sad | Real: happy
Predict: happy | Real: angry
Predict: sad | Real: sad
Predict: happy | Real: angry
Predict: sad | Real: angry
Predict: happy | Real: angry
Predict: disgust | Real: angry
Predict: angry | Real: angry
Predict: surprise | Real: fear
Predict: surprise | Real: surprise
Predict: angry | Real: fear
